In [ ]:
import os

# Set your keys here
os.environ["SERPAPI_KEY"] = "53ccccf615adbd623e0cfc185cd762bcc2978422c94b5a547803cfcc53746bed"
os.environ["COHERE_KEY"] = "hSO3kGpQgR4LOPely08gm7rUVfWbg5G3sVi9Kl5f"



In [ ]:
!pip install aiohttp


In [ ]:
import aiohttp
import asyncio
import json

# Read API keys from environment variables
SERPAPI_KEY = os.getenv("SERPAPI_KEY")
COHERE_KEY = os.getenv("COHERE_KEY")

async def fetch_search_results(query):
    url = "https://serpapi.com/search"
    params = {
        "q": query,
        "api_key": SERPAPI_KEY,
        "engine": "google"
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            return await resp.json()

async def summarize_with_cohere(text):
    url = "https://api.cohere.ai/v1/chat"
    headers = {
        "Authorization": f"Bearer {COHERE_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "command-r",
        "message": f"Summarize the following for a developer:\n{text}"
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=payload) as resp:
            data = await resp.json()
            print("DEBUG Cohere response:", data)  # Optional
            return data.get("text", "No summary available")



async def web_search_agent(query):
    search_data = await fetch_search_results(query)
    if "organic_results" in search_data:
        snippets = [item.get("snippet", "") for item in search_data["organic_results"]]
        combined_text = " ".join(snippets[:5])
        summary = await summarize_with_cohere(combined_text)
        return {
            "query": query,
            "summary": summary.strip(),
            "top_links": [item.get("link") for item in search_data["organic_results"][:5]]
        }

async def main():
    query = "Python async programming best practices"
    result = await web_search_agent(query)
    print(json.dumps(result, indent=2))

await main()



DEBUG Cohere response: {'response_id': '30defb9c-84eb-40cf-b547-3ed8b1862fce', 'text': "For developers interested in asynchronous programming in Python, here's a summary of the key takeaways from the provided text:\n\n- Asyncio Best Practices:\n    - Prefer using async/await over traditional callbacks for better code readability.\n    - Avoid creating functions with long-running loops to prevent blocking the event loop.\n    - Leverage tasks and parallel execution to manage asynchronous operations efficiently.\n\n- Framework Selection:\n    - Choosing the right framework is crucial. Python offers various options like Tornado, asyncio, and third-party options like Sanic. Consider project requirements and use cases when selecting a framework.\n\n- Understanding the Event Loop:\n    - Gaining a deep understanding of the event loop mechanism is essential for writing effective async code. It's the heart of asynchronous programming, managing all pending tasks.\n\n- Async and Await Usage:\n  

In [ ]:
# 📌 New Colab Cell: User Prompt + Short/Long Summary Mode

import os
import aiohttp
import asyncio
import json

# Make sure you have set these API keys earlier in your Colab session
# os.environ["SERPAPI_KEY"] = "your_serpapi_key"
# os.environ["COHERE_KEY"] = "your_cohere_key"

SERPAPI_KEY = os.getenv("SERPAPI_KEY")
COHERE_KEY = os.getenv("COHERE_KEY")

# Function to fetch search results from SerpAPI
async def fetch_search_results(query):
    url = "https://serpapi.com/search"
    params = {
        "q": query,
        "api_key": SERPAPI_KEY,
        "engine": "google"
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            return await resp.json()

# Function to summarize using Cohere Chat API
async def summarize_with_cohere(text, style="long"):
    prompt_type = {
        "long": f"Summarize the following for a developer in detail:\n{text}",
        "short": f"Summarize the following for a developer in 2-3 concise sentences:\n{text}"
    }

    url = "https://api.cohere.ai/v1/chat"
    headers = {
        "Authorization": f"Bearer {COHERE_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "command-r",
        "message": prompt_type[style]
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=payload) as resp:
            data = await resp.json()
            return data.get("text", "No summary available")

# Main function to run search + summaries
async def web_search_agent(query):
    search_data = await fetch_search_results(query)
    if "organic_results" not in search_data:
        return {"error": "No results found"}

    snippets = [item.get("snippet", "") for item in search_data["organic_results"]]
    combined_text = " ".join(snippets[:5])

    long_summary = await summarize_with_cohere(combined_text, style="long")
    short_summary = await summarize_with_cohere(combined_text, style="short")

    return {
        "query": query,
        "short_summary": short_summary.strip(),
        "long_summary": long_summary.strip(),
        "top_links": [item.get("link") for item in search_data["organic_results"][:5]]
    }

# 🔹 Ask user for prompt in Colab
user_query = input("Enter your search query: ")

# Run async function in Colab-safe way
import nest_asyncio
nest_asyncio.apply()

result = await web_search_agent(user_query)
print(json.dumps(result, indent=2))



Enter your search query: TODAY'S VADODARA WEATHER
{
  "query": "TODAY'S VADODARA WEATHER",
  "short_summary": "In Vadodara, Gujarat, India, the next ten days are forecast to be mostly warm and rainy, with temperatures fluctuating between 76\u00b0 and 83\u00b0. There's a chance of flash floods, and today's 'RealFeel' temperature is 85\u00b0. Winds are around 11 mph, and humidity is high at 91%.",
  "long_summary": "For the next ten days, expect mostly cloudy skies and humid conditions in Vadodara, Gujarat, India. Today's temperature will reach a high of 83\u00b0F and drop to a low of 76\u00b0F, with a 78\u00b0 dew point\u2014it's going to feel pretty sticky. There's rain expected today and over the weekend, with a potential for flash floods. \n\nMonday and Tuesday see a continued risk of rain, with possibly a brief shower or two in the morning. The temperatures will remain consistent, holding at 76\u00b0F for the lows and peaking at 77\u00b0F on Wednesday. \n\nLater in the week, the day

In [ ]:
!pip install gradio nest_asyncio aiohttp


In [ ]:
import gradio as gr
import asyncio
import aiohttp
import json
import os

# -------------------------
# Fake User Database (JSON)
# -------------------------
USERS_FILE = "users.json"

def load_users():
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, "r") as f:
            return json.load(f)
    return {}

def save_users(users):
    with open(USERS_FILE, "w") as f:
        json.dump(users, f)

users = load_users()

# -------------------------
# Async Backend (your agent)
# -------------------------
SERPAPI_KEY = os.getenv("SERPAPI_KEY")   # set with: os.environ["SERPAPI_KEY"] = "your_key"
COHERE_KEY = os.getenv("COHERE_KEY")

async def fetch_search_results(query):
    url = "https://serpapi.com/search"
    params = {"q": query, "api_key": SERPAPI_KEY, "engine": "google"}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as resp:
            return await resp.json()

async def summarize_with_cohere(text, style="long"):
    prompt_type = {
        "long": f"Summarize the following for a developer in detail:\n{text}",
        "short": f"Summarize the following for a developer in 2-3 concise sentences:\n{text}"
    }

    url = "https://api.cohere.ai/v1/chat"
    headers = {"Authorization": f"Bearer {COHERE_KEY}", "Content-Type": "application/json"}
    payload = {"model": "command-r", "message": prompt_type[style]}

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=payload) as resp:
            data = await resp.json()
            return data.get("text", "No summary available")

async def web_search_agent(query, mode="short"):
    search_data = await fetch_search_results(query)
    if "organic_results" not in search_data:
        return "❌ No results found", []

    snippets = [item.get("snippet", "") for item in search_data["organic_results"]]
    combined_text = " ".join(snippets[:5])

    summary = await summarize_with_cohere(combined_text, style=mode.lower())
    links = [item.get("link") for item in search_data["organic_results"][:5]]

    return summary.strip(), links

# -------------------------
# Gradio Frontend
# -------------------------
current_user = {"username": None}

def signup(username, password):
    users = load_users()
    if username in users:
        return "❌ Username already exists"
    users[username] = {"password": password}
    save_users(users)
    return "✅ Account created! Please login."

def login(username, password):
    users = load_users()
    if username in users and users[username]["password"] == password:
        current_user["username"] = username
        return f"✅ Welcome {username}! You can now search."
    return "❌ Invalid username or password"

async def run_search(query, mode):
    if not current_user["username"]:
        return "⚠️ Please login first", []
    summary, links = await web_search_agent(query, mode)
    return summary, links

# -------------------------
# Build Gradio UI
# -------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🔎 Web Search Agent with Signup/Login")

    with gr.Tab("Signup"):
        su_user = gr.Textbox(label="Username")
        su_pass = gr.Textbox(label="Password", type="password")
        signup_btn = gr.Button("Signup")
        signup_msg = gr.Textbox(label="Message")
        signup_btn.click(fn=signup, inputs=[su_user, su_pass], outputs=signup_msg)

    with gr.Tab("Login"):
        li_user = gr.Textbox(label="Username")
        li_pass = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        login_msg = gr.Textbox(label="Message")
        login_btn.click(fn=login, inputs=[li_user, li_pass], outputs=login_msg)

    with gr.Tab("Search"):
        query = gr.Textbox(label="Enter search query")
        mode = gr.Radio(["short", "long"], value="short", label="Summary Mode")
        search_btn = gr.Button("Run Search")
        output_summary = gr.Textbox(label="Summary")
        output_links = gr.JSON(label="Top Links")
        search_btn.click(fn=run_search, inputs=[query, mode], outputs=[output_summary, output_links])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3aa9fa06b61aee75fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
